<a href="https://colab.research.google.com/github/VishnuPrem/cnn_action_recognition/blob/master/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Importing the libraries required
from google.colab import drive
!ls "/content/gdrive/My Drive/CIS 519 project"
from google.colab.patches import cv2_imshow
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math
import time

drive.mount('/content/gdrive') #mounting onto Google drive 


ls: cannot access '/content/gdrive/My Drive/CIS 519 project': No such file or directory
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#Function to resize the images/frames
def imagesresize(images):
  print("Resizing all images")
  resized_images=[] 
  for image in images: 
    dim=(100,100)
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA) #resizing all images to given dimensions
    resized_images.append(resized)
  return(resized_images)

#Function to crop the images/frames
def crop_frame(images):
  print("Cropping all images")
  cropped_images=[]
  for im in images:
    w, h = im.shape[:2]   #retrieving dimensions of the image
    w=int(w)
    h=int(h)

    #checking if the width is greater or equal to the height of image and taking appropriate measure (making a square shape)
    if(w==h):
      cropped_im = im #keeping image the same size
    elif(w<h): 
      diff=math.floor((h-w)/2)
      cropped_im = im[0:w,int(diff):h-int(diff)] #cropping image - we are capturing the center of frame. ( for eg h=10,w=6, then for h it is made 2:8 )
    else:
      diff=math.floor((w-h)/2)
      cropped_im = im[int(diff):w-int(diff),0:h] #cropping image
    wn, hn = cropped_im.shape[:2]
    cropped_images.append(cropped_im) #appending the cropped image
  return(cropped_images)

#Function to extract the frames from the video in the specified path
def extract_frames(path,start_time=3,num_frames=10,end_time=4):
    print("Extracting all frames")
    vidObj = cv2.VideoCapture(path) #loading the video from the given path
    count = 0
    imagelist=[]
    success = 1
    
    fps=vidObj.get(cv2.CAP_PROP_FPS)
    frame_count = int(vidObj.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count/fps #Calculating the duration of the video by the total no. of frames/FPS 
    
    start_time=int(duration/2) #Setting the start and end time of the video to give a time of focus
    end_time=start_time+1

    i=start_time*1000 #Converting to milliseconds
    step=int(1000/num_frames) #calculating the step size i.e. the next duration at which the frame should be taken
    while success and i<(end_time*1000): 
        vidObj.set(cv2.CAP_PROP_POS_MSEC,i) #setting the video to the time at which we require the frame
        
        success, image = vidObj.read() #reading the frame in the form of image
        
        image=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) #converting to black and white
        i+=step #Updating the variable to the next time frame
        imagelist.append(image) #appending to the list of images/frames

        #Code to write image at a specific path
        #cv2.imwrite("testpic\\frame%d.jpg" % count, image) 
        #count += 1
    return(imagelist)

#Function used to call the individual functions to pre-process data and return the optical flow stack of the video
def preprocessdata(path, mode = 'OF'):
  '''
  mode 'OF': returns first frame + opticalflow stack
  mode 'IM': returns images stack
  '''
  images = extract_frames(path)
  images = crop_frame(images)
  images = imagesresize(images)

  if mode == 'OF':
    opt_stack = optical_flow_stack(images)
    return opt_stack   # return optical stack with the first rgb image of 1 particular video
  elif mode == 'IM':
    img_stack = image_stack(images)
    return img_stack
  else:
    print("INVALID MODE")

In [0]:
#converts list of images into np array of shape nxnx10
def image_stack(images):
  
  first = True

  for img in images:
    if first:
      img.resize(img.shape[0],img.shape[1],1)
      image_stack = img
      #print(image_stack.shape)
      first = False
    else:
      img.resize(img.shape[0],img.shape[1],1)
      image_stack =  np.concatenate((image_stack,img),axis = 2)
      #print(image_stack.shape)
  
  #print(image_stack.shape)
  return image_stack


In [0]:
# stacks first image with optical flow for all frames

def optical_flow_stack(images):
    
    def optical_flow(prvs, nxt):
      # prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
      # nxt = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
      flow = cv2.calcOpticalFlowFarneback(prvs,nxt, None, 0.5, 3, 5, 3, 5, 1.2, 0)  #(size: (m, n, 2), <type 'numpy.ndarray'>)
      return flow

    first = True 
    prv = images[0]
    #print(images[0].shape)
    #print(len(images))

    #build optical flow stack
    for im in images[1:]:
        nxt = im
        opt_flow = optical_flow(prv,nxt)
        #print('OF:',opt_flow.shape)
        if first:
            #opt_flow.resize(opt_flow.shape[0],opt_flow.shape[1],1)
            opt_flow_stack = opt_flow
            
            first = False
        else:
            #opt_flow.resize(opt_flow.shape[0],opt_flow.shape[1],1)
            opt_flow_stack = np.concatenate((opt_flow_stack,opt_flow),axis = 2)
        #print('Stack:',opt_flow_stack.shape)
        prv = nxt  
    
    #print(opt_flow_stack.shape)
    images[0].resize(images[0].shape[0],images[0].shape[1],1)
    # merge rgb with optical flow stack
    opt_flow_stack =  np.concatenate((images[0],opt_flow_stack),axis = 2)
    #print(opt_flow_stack.shape)
    return opt_flow_stack   

In [0]:
#Function to get the training data and save it into drive under 'x.npy' and 'y.npy'
def process_and_save_data(start,end,im):
  y=[]
  x=[]

  #Extracting the violence videos and appending to the 'x' array, and appending '1' to the 'y' nparray 
  for i in range(start,end+1):
    print("Violence Video: "+str(i))
    optflow=preprocessdata("/content/gdrive/My Drive/CIS 519 project/Videos/Violence/V_"+str(i)+".mp4",im)
    x.append(optflow)
    y.append(1)
  #v_y=np.ones(end-start+1)

  #Extracting the non-violence videos and appending to the 'x' array, and appending '0' to the 'y' nparray
  for i in range(start,end+1):
    print("Non-Violence Video: "+str(i))
    optflow=preprocessdata("/content/gdrive/My Drive/CIS 519 project/Videos/NonViolence/NV_"+str(i)+".mp4",im)
    x.append(optflow)
    y.append(0)
  #nv_y=np.zeros(end-start+1)

  #reshaping the 'y' array and converting the 'x' array to a numpy array
  x=np.array(x)
  #y=np.hstack((v_y,nv_y))
  y=np.array(y).reshape(len(y),1)

  return(x,y)

In [0]:
#Function to save the data onto the defined filepath as a .npy file
def savemethod(data,name):
    outfile = "/content/gdrive/My Drive/CIS 519 project/Model/"+name+".npy" 
    b=np.save(outfile,data)
  
#Loading the specified file into the program
def get_data(filename):
  temp=np.load("/content/gdrive/My Drive/CIS 519 project/Model/"+filename+".npy")
  return(temp)

In [0]:
#Main function
if __name__ == '__main__':
    start_time = time.time() #Starting the timer from the point the program started
    x,y=process_and_save_data(1,1000,'IM') #Processing and loading the required test and train data
    
    savemethod(x,"ImageStack/x_1000")
    savemethod(y,"ImageStack/y_1000")

    x=get_data("ImageStack/x_1000") #Retrieving the data from the saved locations
    y=get_data("ImageStack/y_1000")
    print(x.shape)
    print(y.shape)
    print("--- %s seconds ---" % (time.time() - start_time)) #Time taken for the program to run

Violence Video: 1
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 2
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 3
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 4
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 5
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 6
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 7
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 8
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 9
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 10
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 11
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 12
Extracting all frames
Cropping all images
Resizing all images
Violence Video: 13
Extracting all fra

In [0]:
'''im=cv2.imread("/content/gdrive/My Drive/CIS 519 project/Test_Frames/frame4.jpg")
print(im.shape)'''

def concat(image,optflow):
  ct=np.concatenate((image,optflow),axis=2)
  print(ct.shape)
  return(ct)

def splitdata(ct):
  image=ct[...,0:3]
  print(image.shape)
  l=len(ct[0][0])
  optflow=ct[...,3:l+1]
  print(optflow.shape)
  return(image,optflow)


In [0]:
from sklearn.model_selection import train_test_split
x=get_data("OpticalStack/x_1000") #Retrieving the data from the saved locations
y=get_data("OpticalStack/y_1000")
print(x.shape)
print(y.shape)

x_train, x_vtest, y_train, y_vtest = train_test_split(x, y, test_size = 0.3, random_state = 0)
savemethod(x_train,"OpticalStack/x_train")
print(x_train.shape)
savemethod(y_train,"OpticalStack/y_train")
print(y_train.shape)

print(x_vtest.shape)
print(y_vtest.shape)

x_valid, x_test, y_valid, y_test = train_test_split(x_vtest, y_vtest, test_size = 0.5, random_state = 0)
savemethod(x_valid,"OpticalStack/x_valid")
print(x_valid.shape)
savemethod(y_valid,"OpticalStack/y_valid")
print(y_valid.shape)
savemethod(x_test,"OpticalStack/x_test")
print(x_test.shape)
savemethod(y_test,"OpticalStack/y_test")
print(y_test.shape)


(2000, 100, 100, 19)
(2000, 1)
(1400, 100, 100, 19)
(1400, 1)
(600, 100, 100, 19)
(600, 1)
(300, 100, 100, 19)
(300, 1)
(300, 100, 100, 19)
(300, 1)
